In [106]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import random


In [91]:
def safe_request_loop(url, headers):
    attempt = 0
    while True:
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                print('Nessun errore')
                return response
            else:
                print(f"Errore {response.status_code} - ritento...")
        except Exception as e:
            print(f"Eccezione nella richiesta: {e} - ritento...")

        # Attendi prima di riprovare
        wait_time = min(60, 2 ** attempt + random.uniform(0, 2))
        print(f"Aspetto {wait_time:.2f} secondi prima del tentativo successivo...")
        time.sleep(wait_time)
        attempt += 1

In [92]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "it-IT,it;q=0.9,en-US;q=0.8,en;q=0.7",
    "Referer": "https://www.google.com/"
}
url = 'https://www.transfermarkt.it/serie-a/spieltag/wettbewerb/IT1/plus/?saison_id=2024&spieltag=2'
response = requests.get(url, headers=headers)

In [93]:

soup = BeautifulSoup(response.content, 'html.parser')

section = soup.find('div', class_='large-8 columns')

boxes = section.find_all("div", class_ = "box")


In [118]:
match_data = []
goal_data = []

for box in boxes:
    # Salta box che non contengono risultati
    if not box.find("span", class_="matchresult"):
        continue

    try:
        # Squadre e risultato
        home_team_box = box.find("td", class_ = 'rechts hauptlink no-border-rechts hide-for-small spieltagsansicht-vereinsname')
        #home_team = home_team_box.find("a", href = True).text.strip()

        if home_team_box:
            for a in home_team_box.find_all("a", href=True):
                if a.get("title") != "Discutere della giornata" and a.text.strip():
                    home_team = a.text.strip()
                    break
        away_team_box = box.find("td", class_ = 'hauptlink no-border-links no-border-rechts hide-for-small spieltagsansicht-vereinsname')
        away_team = away_team_box.find("a", href = True).text.strip()

        result = box.find("span", class_="matchresult").text.strip()

        match_data.append({
            "Squadra Casa": home_team,
            "Squadra Trasferta": away_team,
            "Risultato Finale": result
        })

        # Estrai tutti i goal
        events = box.find_all("tr", class_="spieltagsansicht-aktionen")
        
        tds = box.find_all('td', class_='spieltagsansicht')

        for td in tds:
            # Estrai il nome del giocatore
            player_span = td.find('span', class_='hide-for-small')
            if player_span and player_span.find('a'):
                player_name = player_span.find('a').text.strip()
            else:
                continue  # Salta se non trova il nome
            
            # Estrai il minuto del gol dal titolo dell'icona
            goal_icon = td.find('span', class_='icon-tor-formation') or td.find('span', class_='icon-elfmeter-formation')
            if goal_icon and 'title' in goal_icon.attrs:
                minute_info = goal_icon['title']
                # Estrai il numero dal titolo, es. '2.min: gol'
                minute = minute_info.split('.')[0]
            else:
                continue  # Salta se non trova l'icona del gol
            
            print(f"Giocatore: {player_name}, Minuto: {minute}, Squadra casa: {home_team}, Squadra trasferta: {away_team}")

    except Exception as e:
        print(f"Errore durante l'elaborazione di una partita: {e}")

# Crea i DataFrame
df_partite = pd.DataFrame(match_data)
df_goal = pd.DataFrame(goal_data)

print("=== Partite ===")
print(df_partite)
print("\n=== Gol ===")
print(df_goal)

Giocatore: Dennis Man, Minuto: 2, Squadra casa: Parma, Squadra trasferta: Milan
Giocatore: Christian Pulisic, Minuto: 66, Squadra casa: Parma, Squadra trasferta: Milan
Giocatore: Matteo Cancellieri, Minuto: 77, Squadra casa: Parma, Squadra trasferta: Milan
Giocatore: Lorenzo Lucca, Minuto: 5, Squadra casa: Udinese, Squadra trasferta: Lazio
Giocatore: Florian Thauvin, Minuto: 49, Squadra casa: Udinese, Squadra trasferta: Lazio
Giocatore: Gustav Isaksen, Minuto: 90, Squadra casa: Udinese, Squadra trasferta: Lazio
Giocatore: Matteo Darmian, Minuto: 5, Squadra casa: Inter, Squadra trasferta: Lecce
Giocatore: Hakan Çalhanoğlu, Minuto: 69, Squadra casa: Inter, Squadra trasferta: Lecce
Giocatore: Andrea Pinamonti, Minuto: 45, Squadra casa: Monza, Squadra trasferta: Genoa
Giocatore: Mateo Retegui, Minuto: 26, Squadra casa: Torino, Squadra trasferta: Atalanta
Giocatore: Ivan Ilić, Minuto: 31, Squadra casa: Torino, Squadra trasferta: Atalanta
Giocatore: Ché Adams, Minuto: 49, Squadra casa: Torin

In [117]:
import requests
from bs4 import BeautifulSoup

url = "https://www.transfermarkt.it/serie-a/spieltag/wettbewerb/IT1/plus/?saison_id=2024&spieltag=2"
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, "html.parser")

# Trova tutti i td con la classe 'spieltagsansicht'
tds = soup.find_all('td', class_='spieltagsansicht')

for td in tds:
    # Estrai il nome del giocatore
    player_span = td.find('span', class_='hide-for-small')
    if player_span and player_span.find('a'):
        player_name = player_span.find('a').text.strip()
    else:
        continue  # Salta se non trova il nome

    # Estrai il minuto del gol dal titolo dell'icona
    goal_icon = td.find('span', class_='icon-tor-formation') or td.find('span', class_='icon-elfmeter-formation')
    if goal_icon and 'title' in goal_icon.attrs:
        minute_info = goal_icon['title']
        # Estrai il numero dal titolo, es. '2.min: gol'
        minute = minute_info.split('.')[0]
    else:
        continue  # Salta se non trova l'icona del gol

    print(f"Giocatore: {player_name}, Minuto: {minute}")


Giocatore: Dennis Man, Minuto: 2
Giocatore: Christian Pulisic, Minuto: 66
Giocatore: Matteo Cancellieri, Minuto: 77
Giocatore: Lorenzo Lucca, Minuto: 5
Giocatore: Florian Thauvin, Minuto: 49
Giocatore: Gustav Isaksen, Minuto: 90
Giocatore: Matteo Darmian, Minuto: 5
Giocatore: Hakan Çalhanoğlu, Minuto: 69
Giocatore: Andrea Pinamonti, Minuto: 45
Giocatore: Mateo Retegui, Minuto: 26
Giocatore: Ivan Ilić, Minuto: 31
Giocatore: Ché Adams, Minuto: 49
Giocatore: Giovanni Di Lorenzo, Minuto: 45
Giocatore: Khvicha Kvaratskhelia, Minuto: 75
Giocatore: Giovanni Simeone, Minuto: 90
Giocatore: Emmanuel Gyasi, Minuto: 45
Giocatore: Lorenzo Colombo, Minuto: 61
Giocatore: Eldor Shomurodov, Minuto: 80
Giocatore: Roberto Piccoli, Minuto: 44
Giocatore: Patrick Cutrone, Minuto: 53
Giocatore: Dušan Vlahović, Minuto: 28
Giocatore: Nicolò Savona, Minuto: 39
Giocatore: Dušan Vlahović, Minuto: 53


WebDriverException: Message: Process unexpectedly closed with status 0


In [99]:
df_partite

,Squadra Casa,Squadra Trasferta,Risultato Finale
0,Parma,Milan,2:1
1,Udinese,Lazio,2:1
2,Inter,Lecce,2:0
3,Monza,Genoa,0:1
4,Fiorentina,Venezia,0:0
5,Torino,Atalanta,2:1
6,Napoli,Bologna,3:0
7,Roma,Empoli,1:2
8,Cagliari,Como,1:1
9,Verona,Juventus,0:3


In [89]:
boxes[1]

<div class="box" style="border-top: 0 !important;">
<table style="border-top: 0 !important;">
<tbody>
<tr class="table-grosse-schrift">
<td class="rechts hauptlink no-border-rechts hide-for-small spieltagsansicht-vereinsname">
<span class="tabellenplatz">(12.)</span> 
                                                                    <a href="/parma-calcio-1913/spielplan/verein/130/saison_id/2024" title="Parma Calcio">Parma</a> </td>
<td class="rechts hauptlink no-border-rechts show-for-small spieltagsansicht-vereinsname">
<span class="tabellenplatz">(12.)</span> 
                                                                    <a href="/parma-calcio-1913/spielplan/verein/130/saison_id/2024" title="Parma Calcio">Parma</a> </td>
<td class="hauptlink zentriert no-border-links no-border-rechts hide-for-small spieltagsansicht-wappen">
<a href="/parma-calcio-1913/spielplan/verein/130/saison_id/2024" title="Parma Calcio"><img alt="Parma Calcio" class="" src="https://tmssl.akamaized.net//